In [2]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = 'optical_flow_images'

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def track_features(self):
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)
        frame_count = 0

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

        self.cap.release()
        cv2.destroyAllWindows()

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def get_optical_images(self):
        os.makedirs(self.output_folder)
        self.get_video_path()
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)
        frame_count = 0

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{frame_count:04d}.png'), flow_image)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

        self.cap.release()
        cv2.destroyAllWindows()

    def show_optical_flow_video(self):
        self.get_video_path()
        self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    feature_tracker = LucasKanade_OpticalFlow()

    def get_video_path():
        feature_tracker.get_video_path()

    def get_optical_images():  # Fix: Add a function for the "Get Optical Images" button
        feature_tracker.get_optical_images()

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    # Buttons
    btn_get_video_path = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_video_path.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()
